In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pandas as pd
import ruptures as rpt
import seaborn as sns

from tqdm import tqdm

### Data Loading

In [ ]:
BIKE_COUNT_URLS_AND_LOAD_KWARGS = [
    ("https://at.govt.nz/media/1991027/auckland-trasnport-cycle-counts-jan-dec-2022.xlsx", {}),
    ("https://at.govt.nz/media/1991376/auckland-transport-january-2023-cycle-counts.xlsx", {}),
    ("https://at.govt.nz/media/1991377/auckland-transport-february-2023-cycle-counts.xlsx", {}),
    ("https://at.govt.nz/media/1991436/auckland-transport-march-2023-cycle-counts.xlsx", {"na_values": "Pending"}),
    ("https://at.govt.nz/media/1992545/april-2023-cycle-counts.xlsx", {}),
    ("https://at.govt.nz/media/1992547/may-2023-cycle-counts.xlsx", {}),
    ("https://at.govt.nz/media/1992546/june-2023-cycle-counts.xlsx", {}),
    ("https://at.govt.nz/media/xs5lybun/at-daily-cycle-data-july-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/t3zlbdn4/at-daily-cycle-data-august-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/53rj40ji/at-daily-cycle-data-september-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/hl0jbsrl/at-daily-cycle-data-october-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/ty4js45t/at-daily-cycle-counts-november-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/1tvdgsgo/at-daily-cycle-counts-december-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/bb4h3wd3/at-daily-cycle-counts-january-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/xlcaru0v/at-daily-cycle-counts-feb-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/4g3hzpp5/at-daily-cycle-counts-march-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/htvezqdn/at-daily-cycle-counts-april-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/ue5cygl0/at-daily-cycle-counts-may-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/3icd2jug/at-daily-cycle-counts-june-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/jbdd1rox/cycle-counts-july-2024.xlsx", {"header": 2, "na_values": "z"}),
    ("https://at.govt.nz/media/bvadzmqg/cycle-counts-august-2024.xlsx", {"usecols": "G:CG"}),
    ("https://at.govt.nz/media/lpsfdwbe/auckland-transport-cycle-counts-september-2024.xlsx", {"usecols": "G:CG"}),
    ("https://at.govt.nz/media/ohbhvmrl/auckland-transport-cycle-movements-october-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/chelni1f/auckland-transport-cycle-movements-data-november-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/zdumuud2/auckland-transport-cycle-movements-december-2024.xlsx", {"header": 2}),
]

In [ ]:
LOCATION_MAPPING = {
    "GI TO TAMAKI DR SECTION-1": "GI to Tamaki Drive Section-1",
    "Great North Road NB Towards CBD": "Great North Road",
    "Great South Road Manukau": "Great South Road",
    "Highbrook Pathway": "Highbrook Drive",
    "Lake Road Total New": "Lake Road",
    "Leigh Road Shared Path": "Leigh Road",
    "Mangere Foot Bridge": "Mangere Bridge",
    "Ocean View Road": "Oceanview Road",
    "Rathgar Road": "Rathger Road",
    "Remuera Road": "Remurua Road",
    "SH18 Upper Harbour Drive": "Upper Harbour",
    "SH20A Shared Path": "SH20A",
    "SW Shared Path": "SW SH20",
    "Symonds Street Total": "Symonds Street",
    "Tamaki Drive - Parnell": "Tamaki Drive",
    "TeAtatu Peninsula Shared Path": "TeAtatu Peninsula",
    "TeWero Bridge Bike Counter": "TeWero Bridge",
    "Ti Rakau - Opposite Bunnings": "Ti Rakau",
    "Ti Rakau opposite 92": "Ti Rakau",
    "Twin Streams Shared Path": "Twin Streams",
    "Upper Harbour Shared Path": "Upper Harbour",
    "Waterview Unitec Shared Path": "Waterview Unitec",
}

LOCATIONS_TO_DROP = [
    "Ngapipi Road SUP",
    "Meadowbank SUP",
    "Rankin Ave Shared Path",
    "Ti Rakau",
    "Ti Rakau Reserve",
    "Ti Rakau Riverhills",
    "TeWero Bridge",
]

In [ ]:
def load_cycle_counts(url: str, **read_excel_kwargs) -> pd.DataFrame:
    df = pd.read_excel(url, **read_excel_kwargs)
    df = df.rename(columns={"Date": "date", "Date.1": "date", "Time": "date"})
    df = df.melt(id_vars="date", var_name="location", value_name="count")
    df = df.astype({"date": "datetime64[ns]", "location": str, "count": float})
    return df


def drop_locations(df: pd.DataFrame) -> pd.DataFrame:
    to_drop = df["location"].isin(LOCATIONS_TO_DROP)
    return df.loc[~to_drop]


def map_locations(location: str) -> str:
    return LOCATION_MAPPING.get(location, location)


def standardise_locations(df: pd.DataFrame) -> pd.DataFrame:
    df["location"] = df["location"].str.replace(r" Cyclists?", "", regex=True)
    df["location"] = df["location"].str.replace(r"\bDr\b", "Drive", regex=True)
    df["location"] = df["location"].str.replace(r"\bRd\b", "Road", regex=True)
    df["location"] = df["location"].str.replace(r"\bSt\b", "Street", regex=True)
    df["location"] = df["location"].apply(map_locations)
    return df

In [ ]:
dfs = []
for url, load_kwargs in tqdm(BIKE_COUNT_URLS_AND_LOAD_KWARGS):
    try:
        df = load_cycle_counts(url, **load_kwargs)
    except Exception as e:
        print("Could not load data for url: ", url)
        print("Reason: ", e)
        continue
    dfs.append(df)

cycle_counts = pd.concat(dfs, axis=0, ignore_index=True)
cycle_counts = standardise_locations(cycle_counts)
cycle_counts = drop_locations(cycle_counts)
cycle_counts = cycle_counts.drop_duplicates()

In [ ]:
(
    cycle_counts
    .assign(is_na=cycle_counts["count"].isna())
    .groupby("location")
    [["is_na"]]
    .sum()
    .sort_values("is_na", ascending=False)
    .iloc[:10]
)

In [ ]:
cycle_counts.to_csv("raw_cycle_counts.csv", index=False)

In [ ]:
cycle_counts = pd.read_csv("raw_cycle_counts.csv", parse_dates=["date"])

### Outlier detection and removal

In [ ]:
# Outlier
outlier_locs = [
    "Albany Highway",
    "Archibald Park",
    "GI to Tamaki Drive Section-1",
    "Lake Road",
    "Mangere Safe Routes",
    "NW Cycleway TeAtatu",
    "Sandringham Road",
    "SH20A",
    "SW SH20",
]

In [ ]:
def get_outlier_df(location_df: pd.DataFrame, n_sigma: float = 3.0) -> pd.DataFrame:
    outlier_df = location_df.copy()

    # Compute mean and std by weekday
    outlier_df = outlier_df.assign(weekday=outlier_df["date"].dt.weekday)
    weekday_stats = (
        outlier_df
        .groupby("weekday")[["count"]]
        .agg(["mean", "std"])
        .droplevel(level=0, axis=1)
        .reset_index()
    )
    outlier_df = outlier_df.merge(weekday_stats, how="left", on="weekday")
    
    # Define upper and lower bounds
    outlier_df = outlier_df.assign(
        upper=outlier_df["mean"] + n_sigma * outlier_df["std"],
        lower=outlier_df["mean"] - n_sigma * outlier_df["std"],
    )
    is_outlier = (outlier_df["count"] > outlier_df["upper"]) | (outlier_df["count"] < outlier_df["lower"])
    outlier_df = outlier_df.assign(is_outlier=is_outlier)
    
    return outlier_df


def drop_outliers(
    cycle_counts: pd.DataFrame,
    outlier_dates_and_locs: list[tuple[datetime, str]],
):
    # Construct filter
    to_drop = pd.Series(np.full((len(cycle_counts), ), False), index=cycle_counts.index)
    for date, loc in outlier_dates_and_locs:
        to_drop |= ((cycle_counts["date"] == date) & (cycle_counts["location"] == loc))
    
    return cycle_counts.loc[~to_drop]

In [ ]:
loc = outlier_locs[8]
location_df = cycle_counts[cycle_counts["location"] == loc]

n_sigma = 3.0
outlier_df = get_outlier_df(location_df, n_sigma=n_sigma)
outlier_df = outlier_df.sort_values("date")

fig, ax = plt.subplots(figsize=(10, 3.5))
ax.plot(outlier_df["date"].values, outlier_df["count"].values)
ax.plot(outlier_df["date"].values, outlier_df["upper"].values, ls="--")
ax.plot(outlier_df["date"].values, outlier_df["lower"].values, ls="--")
ax.set(title=loc)

In [ ]:
outlier_dates_and_locations = [
    (datetime(2024, 10, 12), "Albany Highway"),
    (datetime(2024, 10, 21), "Albany Highway"),
    
    (datetime(2023, 4, 12), "Archibald Park"),
    (datetime(2024, 3, 1), "Archibald Park"),
    (datetime(2024, 3, 9), "Archibald Park"),
    (datetime(2024, 9, 28), "Archibald Park"),
    
    (datetime(2022, 5, 28), "GI to Tamaki Drive Section-1"),
    (datetime(2022, 6, 4), "GI to Tamaki Drive Section-1"),
    
    (datetime(2024, 9, 2), "Lake Road"),
    
    (datetime(2022, 10, 15), "Mangere Safe Routes"),
    (datetime(2022, 12, 2), "Mangere Safe Routes"),
    (datetime(2024, 9, 24), "Mangere Safe Routes"),
    
    (datetime(2022, 5, 7), "NW Cycleway TeAtatu"),
    
    (datetime(2024, 2, 19), "Sandringham Road"),
    (datetime(2024, 2, 20), "Sandringham Road"),
    (datetime(2024, 2, 21), "Sandringham Road"),
    (datetime(2024, 2, 22), "Sandringham Road"),
    (datetime(2024, 2, 23), "Sandringham Road"),

    (datetime(2022, 3, 2), "SH20A"),
    (datetime(2024, 2, 22), "SH20A"),

    (datetime(2024, 2, 2), "SW SH20"),
    (datetime(2024, 2, 3), "SW SH20"),
    (datetime(2024, 3, 27), "SW SH20"),
    (datetime(2024, 4, 11), "SW SH20"),
    (datetime(2024, 4, 12), "SW SH20"),
    (datetime(2024, 4, 13), "SW SH20"),
]

In [ ]:
cycle_counts = drop_outliers(cycle_counts, outlier_dates_and_locations)

In [ ]:
cycle_counts.to_csv("cycle_counts_after_outliers.csv", index=False)

### Changepoint detection and adjustment

In [ ]:
def get_chpt_locations(location_df: pd.DataFrame, n_chpts: int):
    location_df = location_df.copy()
    location_df = location_df.sort_values("date")
    
    model = rpt.Dynp(model="l2").fit(np.array(location_df["count"]))
    chpts_idx = model.predict(n_bkps=n_chpts)
    
    min_date, max_date = location_df["date"].min(), location_df["date"].max()
    chpts_dt = [location_df.iloc[idx]["date"].to_pydatetime() for idx in chpts_idx[:-1]]
    chpts_dt = [min_date.to_pydatetime()] + chpts_dt + [max_date.to_pydatetime()]
    
    return chpts_dt


def adjust_scale(
    cycle_counts: pd.DataFrame,
    chpts_by_location: dict[str, list[datetime]],
):
    for loc, chpts in chpts_by_location.items():
        loc_mask = cycle_counts["location"] == loc
        
        # Find the longest streak, this is the anchor
        chpt_streaks = list(zip(chpts, chpts[1:]))
        streak_lengths = [
            (loc_mask & cycle_counts["date"].between(start, end, inclusive="left")).sum()
            for (start, end) in chpt_streaks
        ]
        longest_streak = chpt_streaks[streak_lengths.index(max(streak_lengths))]

        # Calculate mean of longest streak
        ls_start, ls_end = longest_streak
        ls_mask = cycle_counts["date"].between(ls_start, ls_end, inclusive="left")
        ls_mean = cycle_counts[loc_mask & ls_mask]["count"].mean()

        # Scale every streak using anchor stats
        for streak_start, streak_end in chpt_streaks:
            streak_mask = cycle_counts["date"].between(streak_start, streak_end, inclusive="left")
            streak_mean = cycle_counts[loc_mask & streak_mask]["count"].mean()
            loc_streak_counts = cycle_counts.loc[loc_mask & streak_mask, "count"]
            loc_streak_counts_adj = loc_streak_counts.div(streak_mean).mul(ls_mean)
            cycle_counts.loc[loc_mask & streak_mask, "count"] = loc_streak_counts_adj
    
    return cycle_counts

In [ ]:
locations_and_n_chpts = [
    ("Grafton Road", 1),
    ("Great South Road", 2),
    ("Karangahape Road", 2),
    ("Lightpath", 1)
]

In [ ]:
loc, n_chpts = locations_and_n_chpts[1]
location_df = cycle_counts[cycle_counts["location"] == loc].sort_values("date")

chpts = get_chpt_locations(location_df, n_chpts)

fig, ax = plt.subplots(figsize=(12, 2.5))
ax.plot(location_df["date"].values, location_df["count"].values)
for chpt in chpts:
    ax.axvline(chpt, color="red")
ax.set(title=loc)
fig.tight_layout();

In [ ]:
chpts_by_location = {}
for loc, n_chpts in locations_and_n_chpts:
    location_df = cycle_counts[cycle_counts["location"] == loc].sort_values("date")
    chpts = get_chpt_locations(location_df, n_chpts)
    chpts_by_location[loc] = chpts

In [ ]:
chpts_by_location = {
    "Grafton Road": [
        datetime(2022, 1, 1),
        datetime(2023, 1, 16),
        datetime(2024, 12, 31),
    ],
    "Great South Road": [
        datetime(2022, 1, 1),
        datetime(2022, 8, 29),
        datetime(2023, 3, 2),
        datetime(2024, 12, 31)
    ],
    "Karangahape Road": [
        datetime(2022, 1, 1),
        datetime(2024, 4, 15),
        datetime(2024, 7, 19),
        datetime(2024, 12, 31)
    ],
    "Lightpath": [
        datetime(2022, 1, 1),
        datetime(2023, 2, 5),
        datetime(2024, 12, 31)
    ]
}

In [ ]:
cycle_counts = adjust_scale(cycle_counts, chpts_by_location)

In [ ]:
loc, _ = locations_and_n_chpts[3]
location_df = cycle_counts[cycle_counts["location"] == loc].sort_values("date")

fig, ax = plt.subplots(figsize=(12, 2.5))
ax.plot(location_df["date"].values, location_df["count"].values)
for chpt in chpts_by_location[loc]:
    ax.axvline(chpt, color="red")
ax.set(title=loc)
fig.tight_layout();

In [ ]:
i = 3
locs = sorted(cycle_counts["location"].unique())[i * 5:  (i + 1) * 5]
print(locs)

fig, ax = plt.subplots(len(locs), 1, figsize=(10, len(locs) * 1.75), sharex=True)
for i, loc in enumerate(locs):
    location_df = cycle_counts[cycle_counts["location"] == loc].sort_values("date")
    ax[i].plot(location_df["date"].values, location_df["count"].values, label=loc)
    ax[i].legend()

fig.tight_layout();

In [ ]:
cycle_counts.to_csv("cycle_counts_after_scale_adjust.csv", index=False)

### Resample to daily freq

In [ ]:
location_dfs = []
for loc, location_df in cycle_counts.groupby("location"):
    break

In [ ]:
location_df[location_df.duplicated("date")]

In [ ]:
location_df[location_df["date"] == datetime(2024, 8, 31)]

## EDA

### Total counts by day

In [ ]:
daily_total = cycle_counts.groupby("date")[["count"]].sum().reset_index()

fig, ax = plt.subplots(1, 2, figsize=(10, 3.5))

ax[0].plot(daily_total["date"], daily_total["count"])
ax[0].set(ylabel="Count")
for tick in ax[0].get_xticklabels():
    tick.set_rotation(45)

ax[1].hist(daily_total["count"], bins=20);
ax[1].set(xlabel="Count", ylabel="Frequency")

fig.tight_layout()

### Daily counts by location

In [ ]:
av_daily_counts = cycle_counts.groupby("date")[["count"]].mean().reset_index()

fig, ax = plt.subplots(2, 1, figsize=(12, 5), sharex=True)

locations = cycle_counts["location"].unique()
for location in locations:
    outlier_df = cycle_counts[cycle_counts["location"] == location]
    ax[0].plot(outlier_df["date"], outlier_df["count"], color="gray", alpha=0.25)
    ax[1].plot(outlier_df["date"], np.log1p(outlier_df["count"]), color="gray", alpha=0.25)

ax[0].plot(av_daily_counts["date"], av_daily_counts["count"], color="blue", linewidth=2)
ax[1].plot(av_daily_counts["date"], np.log1p(av_daily_counts["count"]), color="blue", linewidth=2)

ax[0].set(ylabel="Count")
ax[1].set(ylabel="Log(Count)")

In [ ]:
av_daily_counts_by_loc = cycle_counts.groupby("location")[["count"]].mean().reset_index()
av_daily_counts_by_loc = av_daily_counts_by_loc.sort_values("count").reset_index(drop=True)

mean_counts = av_daily_counts_by_loc["count"].mean()
median_counts = av_daily_counts_by_loc["count"].median()

fig, ax = plt.subplots(1, 1)
ax.hist(av_daily_counts_by_loc["count"], bins=20)
ax.axvline(mean_counts, color="red", linestyle="--", lw=2.5, label="Mean")
ax.axvline(median_counts, color="orange", linestyle="--", lw=2.5, label="Median")
ax.set(xlabel="Average Daily Count", ylabel="Frequency")
ax.legend();

### High demand locations

In [ ]:
threshold = 600
high_demand_locs = av_daily_counts_by_loc[av_daily_counts_by_loc["count"] > threshold]
high_demand_locs.sort_values("count")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

locations = high_demand_locs["location"].values
colors = cm.coolwarm(np.linspace(0, 1, len(locations)))
for i, location in enumerate(locations):
    outlier_df = cycle_counts[cycle_counts["location"] == location].sort_values("date").iloc[-150:]
    ax.plot(
        outlier_df["date"],
        outlier_df["count"],
        label=location,
        color=colors[i],
        lw=2.5,
    )
ax.set(ylabel="Count")
fig.tight_layout();

In [ ]:
counts_pivot = cycle_counts.pivot_table(columns="date", index="location", values="count", aggfunc="sum")
counts_pivot = counts_pivot.fillna(0).assign(average=lambda x: x.mean(axis=1))
counts_pivot = counts_pivot[counts_pivot["average"].between(400, 1000)]
counts_pivot = counts_pivot.sort_values("average", ascending=False).drop(columns="average")

fig, ax = plt.subplots(1, 1)
sns.heatmap(counts_pivot, cmap="coolwarm", cbar_kws={"label": "Count"})

xs = np.arange(0, counts_pivot.shape[1], 60)
xticklables = [c.strftime("%Y-%m-%d") for c in counts_pivot.columns[xs]]
ax.set(xticks=xs, xticklabels=xticklables)

ys = np.arange(0, counts_pivot.shape[0])
yticklables = counts_pivot.index[ys]
ax.set_yticks(ys + 0.5)
ax.set_yticklabels(yticklables, fontsize="small")

ax.set(xlabel="", ylabel="");